https://gist.github.com/Sentdex/130c225d90acec7c808b8ba5aba0eda1

In [1]:
import torch
from diffusers import StableDiffusionPipeline
from torch import autocast
import random
import matplotlib.pyplot as plt
import os


/home/mal/miniconda3/envs/project-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mal/miniconda3/envs/project-env/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/mal/miniconda3/envs/project-env/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
prompts = [
          "1965 Porsche 911",
          "1975 Porsche 911",
          "1985 Porsche 911",
          "1995 Porsche 911",
          "2005 Porsche 911 front",
          "2015 Porsche 911",
          "2020 Porsche 911",
          "2020 Porsche 911 GT3 RS"]

In [4]:
# make sure you're logged in with `huggingface-cli login`
pipe = StableDiffusionPipeline.from_pretrained(
    'CompVis/stable-diffusion-v1-4', 
    # revision='fp16',
    # torch_dtype=torch.float16, 
    # use_auth_token=True
    torch_dtype=torch.float32,
    low_cpu_mem_usage=True,
    safety_checker = None,
    requires_safety_checker = False,
)

Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00,  7.14it/s]


In [5]:
pipe = pipe.to('cuda')

In [11]:
def infer(prompt,          
    num_inference_steps=10, 
    samples=5,
    seed=1024,
    guidance_scale=7.5, 
    width=512, height=512
):
    generator = torch.Generator("cuda").manual_seed(seed)
    w = width//8*8
    h = height//8*8
    with autocast("cuda"):
        image = pipe(
            prompt, 
            guidance_scale=7.5, 
            generator=generator,
            width=w, height=h, 
            num_inference_steps=num_inference_steps)[0]
    return image


In [ ]:
# for p in prompts:

#     prompt_orig = p 
#     if not os.path.exists("imagery"):
#         os.mkdir("imagery")

#     if not os.path.exists(f"imagery/{prompt_orig}"):
#         os.mkdir(f"imagery/{prompt_orig}/")

#     HM = 200
#     for i in range(HM):
#         print(f"{i+1}/{HM}")
#         prompt_to_use = prompt_orig
#         seed = random.randint(0, 10000)
#         print(seed)
#         image = infer(prompt_to_use, num_inference_steps=75, 
#                       seed=seed, width=512, height=512)
#         image.save(f"imagery/{prompt_to_use}/{seed}.png")

In [12]:
from datetime import datetime

for p in prompts:

    HM = 200
    
    for i in range(HM):
        
        print(f"{i+1}/{HM}")

        seed = random.randint(0, 10000)

        image = infer(
            p, 
            num_inference_steps=75, 
            seed=seed, 
            width=512, 
            height=512
        )
        
        milis = int(datetime.now().timestamp() * 1000)

        image.save(f"../../images/stabilityai/sd_v21/out/{milis}{p}/{seed}.png")

1/200


 37%|███▋      | 28/75 [04:35<07:43,  9.87s/it]